In [1]:
import pandas as pd  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [2]:
dataset = pd.read_csv("../data/raw/titanic.csv")

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
dataset["Ticket"].value_counts()

Ticket
347082      7
CA. 2343    7
1601        7
3101295     6
CA 2144     6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: count, Length: 681, dtype: int64

In [5]:
dataset.groupby("Cabin").agg({"Survived": "mean", "Embarked": "count"}).reset_index().sort_values(by="Survived", ascending=False)

,Cabin,Survived,Embarked
73,C62 C64,1.0,1
97,D21,1.0,1
94,D17,1.0,2
95,D19,1.0,1
60,C148,1.0,1
...,...,...,...
67,C46,0.0,1
64,C30,0.0,1
59,C128,0.0,1
56,C124,0.0,2


In [6]:
dataset["Embarked"].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [7]:
embarked = pd.get_dummies(dataset["Embarked"])
gender = pd.get_dummies(dataset["Sex"])
dataset = pd.concat([dataset, embarked, gender], axis=1)


In [8]:
dataset = dataset.drop(["PassengerId", "Name", "Ticket", "Cabin", "Embarked", "Sex"], axis=1)

In [9]:
for col in dataset.select_dtypes(include=["bool"]).columns:
    dataset[col] = dataset[col].astype(int)






In [10]:
dataset = dataset.drop_duplicates()

In [11]:
dataset["Age"] = dataset["Age"].fillna(dataset["Age"].median())

In [12]:
dataset.isna().sum()

Survived    0
Pclass      0
Age         0
SibSp       0
Parch       0
Fare        0
C           0
Q           0
S           0
female      0
male        0
dtype: int64

In [21]:
def preprocess(dataset_path: str):
    dataset = pd.read_csv("../data/raw/titanic.csv")
    embarked = pd.get_dummies(dataset["Embarked"])
    gender = pd.get_dummies(dataset["Sex"])
    dataset = pd.concat([dataset, embarked, gender], axis=1)
    dataset = dataset.drop(["PassengerId", "Name", "Ticket", "Cabin", "Embarked", "Sex"], axis=1)
    for col in dataset.select_dtypes(include=["bool"]).columns:
        dataset[col] = dataset[col].astype(int)
    dataset = dataset.drop_duplicates()
    dataset["Age"] = dataset["Age"].fillna(dataset["Age"].median())
    return dataset


In [22]:
dataset = preprocess("../data/raw/titanic.csv")

In [23]:
X, y = dataset.drop(["Survived"], axis=1), dataset["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.79      0.82      0.81        91
           1       0.74      0.69      0.71        65

    accuracy                           0.77       156
   macro avg       0.76      0.76      0.76       156
weighted avg       0.77      0.77      0.77       156

